# 计算性能

## 命令式和符号式混合编程

In [1]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

fancy_func(1, 2, 3, 4)

10

PyTorch使用的是命令式编程，故下面略。

## 异步计算

默认情况下，PyTorch中的 GPU 操作是异步的。当调用一个使用 GPU 的函数时，这些操作会在特定的设备上排队但不一定会在稍后立即执行。这就使我们可以并行更多的计算，包括 CPU 或其他 GPU 上的操作。 

一般情况下，异步计算的效果对调用者是不可见的，因为

1. 每个设备按照它们排队的顺序执行操作

2. CPU 和 GPU 之间或两个 GPU 之间复制数据时，PyTorch会自动执行必要的同步操作。因此，计算将按每个操作同步执行的方式进行。 可以通过设置环境变量CUDA_LAUNCH_BLOCKING = 1来强制进行同步计算。当 GPU 产生error时，这可能非常有用。（异步执行时，只有在实际执行操作之后才会报告此类错误，因此堆栈跟踪不会显示请求的位置。）

## 自动并行计算

GPU操作默认是异步的，当调用一个使用GPU的函数时，这些操作会在特定的设备上排队，但不一定会在稍后执行。这允许并行更多的计算，包括CPU或其他GPU上的操作。

In [2]:
import torch
import time

assert torch.cuda.device_count() >= 2

In [3]:
class Benchmark():
    def __init__(self, prefix=None):
        self.prefix = prefix + ' ' if prefix else ''
        
    def __enter__(self):
        self.start = time.time()
        
    def __exit__(self, *args):
        print('%stime: %.4f sec' % (self.prefix, time.time() - self.start))

In [4]:
def run(x):
    for _ in range(20000):
        y = torch.mm(x, x)

In [5]:
x_gpu1 = torch.rand(size=(100, 100), device='cuda:0')
x_gpu2 = torch.rand(size=(100, 100), device='cuda:1')

In [6]:
with Benchmark('Run on GPU1'):
    run(x_gpu1)
    torch.cuda.synchronize()
    
with Benchmark('Run on GPU2'):
    run(x_gpu2)
    torch.cuda.synchronize()

Run on GPU1 time: 1.2626 sec
Run on GPU2 time: 1.1179 sec


In [7]:
with Benchmark('Run on both GPU1 and GPU2 in parallel'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

Run on both GPU1 and GPU2 in parallel time: 1.8384 sec


## 多GPU计算

In [9]:
! nvidia-smi

Thu Oct  8 08:13:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.74       Driver Version: 418.74       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:04:00.0 Off |                    0 |
| N/A   59C    P0    77W / 149W |   7695MiB / 11441MiB |     61%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   47C    P0    80W / 149W |   7220MiB / 11441MiB |     34%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [10]:
import torch
net = torch.nn.Linear(10, 1).cuda()
net

Linear(in_features=10, out_features=1, bias=True)

In [11]:
net = torch.nn.DataParallel(net)
net

DataParallel(
  (module): Linear(in_features=10, out_features=1, bias=True)
)